In [1]:

import warnings

import matplotlib.pyplot as plt
# import MultiTrade
import numpy as np
import pandas as pd
from IPython.display import display
from neuralforecast.core import NeuralForecast

from neuralforecast.auto import AutoBiTCN, AutoNBEATS, AutoTFT
from neuralforecast.losses.pytorch import RMSE

# from statsforecast import StatsForecast

from utils.utils import build_market_image
from configs import defaults
import datetime
warnings.filterwarnings("ignore")

In [2]:
COIN_PAIRS=defaults.COIN_PAIRS
target_pair=defaults.target_pair
time_frame=defaults.time_frame

In [3]:

horizon = 7
context_length=5*horizon
start_dt=pd.Timestamp('2024-01-01')
split_dt=datetime.datetime.now()-pd.Timedelta(days=7)
end_dt=datetime.datetime.now()
split_buffer=pd.Timedelta(unit=time_frame[-1],value=context_length)

In [5]:

data=build_market_image(target_pair=target_pair,time_frame=time_frame,axis=0,data_dir='data_futures')


train_data=data.groupby('symbol').apply(lambda x: x[start_dt:split_dt])
test_data=data.groupby('symbol').apply(lambda x: x[split_dt-split_buffer:end_dt])

train_data=train_data.reset_index(level=0,drop=True).reset_index()
test_data=test_data.reset_index(level=0,drop=True).reset_index()
# data=data[data['symbol']==target_pair].copy()
data['symbol'].unique()
id_col='symbol'


# test_data

['data_futures/coinbase-ETHUSD_Futures-1h.pkl', 'data_futures/coinbase-BTCUSD_Futures-1h.pkl', 'data_futures/coinbase-LTCUSD_Futures-1h.pkl', 'data_futures/coinbase-SUIUSD_Futures-1h.pkl', 'data_futures/coinbase-DOGEUSD_Futures-1h.pkl']


100%|██████████| 78/78 [00:00<00:00, 12198.65it/s]
5it [00:00,  7.71it/s]


KeyError: Timestamp('2024-01-01 00:00:00')

In [5]:
# train_data

In [6]:
front=['y','ds','symbol']
exo_gen_cols=data.filter(like='feature_').columns.tolist()
cols=front+[c for c in exo_gen_cols if c not in front]

train_data=train_data[cols]
test_data=test_data[cols]
# train_data
# data[id_col].unique()
# 

In [7]:
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [8]:
forecast_horizon=6
backend='optuna'
BiTCN_config=AutoBiTCN.get_default_config(h=forecast_horizon,backend=backend)

TFT_config=AutoTFT.get_default_config(h=forecast_horizon,backend=backend)
NBEATS_config=AutoNBEATS.get_default_config(h=forecast_horizon,backend=backend)

In [9]:
BiTCN_MODEL= AutoBiTCN(h=forecast_horizon,
                  loss=RMSE(),
                  config=BiTCN_config,
                  search_alg=optuna.samplers.TPESampler(),
                  backend='optuna',
                  num_samples=10)

TFT_MODEL= AutoTFT(h=forecast_horizon,
                  loss=RMSE(),
                  config=TFT_config,
                  search_alg=optuna.samplers.TPESampler(),
                  backend='optuna',
                  num_samples=10)

NBEATS_MODEL= AutoNBEATS(h=forecast_horizon,
                  loss=RMSE(),
                  config=NBEATS_config,
                  search_alg=optuna.samplers.TPESampler(),
                  backend='optuna',
                  num_samples=10)

models=[BiTCN_MODEL,TFT_MODEL,NBEATS_MODEL]

In [10]:

model = NeuralForecast(models=models,
                    local_scaler_type='robust',
                    
                     freq='1h')

In [11]:
model.fit(train_data,
        val_size=horizon,
        time_col='ds',                    
        target_col='y',
        id_col=id_col,
        
        )


Seed set to 4
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name          | Type          | Params | Mode 
--------------------------------------------------------
0 | loss          | RMSE          | 0      | train
1 | padder_train  | ConstantPad1d | 0      | train
2 | scaler        | TemporalNorm  | 0      | train
3 | lin_hist      | Linear        | 32     | train
4 | drop_hist     | Dropout       | 0      | train
5 | net_bwd       | Sequential    | 5.4 K  | train
6 | drop_temporal | Dropout       | 0      | train
7 | temporal_lin1 | Linear        | 496    | train
8 | temporal_lin2 | Linear        | 102    | train
9 | output_lin    | Linear        | 17     | train
--------------------------------------------------------
6.0 K     Trainable params
0         Non-trainable params
6.0 K     Total params
0.024     Total estimated model params size (MB)
30        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=500` reached.


In [15]:
model.save('MultiHeadForecastingModel/',overwrite=True)

In [ ]:
test_dates=test_data.sort_values('ds')['ds'].unique()
last_date=train_data['ds'].unique()[-1]

In [ ]:
test_data

In [ ]:
fcst_df = model.predict()
plot_df=train_data.groupby('symbol').tail(context_length)
plot_df
# fcst_df.tail()

In [ ]:
from statsforecast import StatsForecast


In [ ]:
StatsForecast.plot(plot_df, fcst_df, engine='matplotlib',id_col=id_col, max_insample_length=context_length,)


In [ ]:
pred_frame=data.groupby('symbol').apply(lambda x: x[last_date-split_buffer:split_dt])
pred_frame=pred_frame.reset_index(level=0,drop=True).reset_index()
pred_frame=pred_frame.drop('date_close',axis=1)

In [ ]:
model=NeuralForecast.load('MultiHeadForecastingModel/')

In [ ]:
fcst_df = model.predict(pred_frame)

# fcst_df.head()

In [ ]:
StatsForecast.plot(pred_frame, fcst_df, engine='matplotlib',id_col=id_col, max_insample_length=context_length,)


In [ ]:
test_data

In [ ]:
# test_data=test_data.drop('close_date',axis=1)
test_data[exo_gen_cols].values.dtype

In [ ]:
model.dataset, model.uids, model.last_dates, model.ds = model._prepare_fit(
                df=test_data,
                static_df=None,
                sort_df=model.sort_df,
                predict_only=False,
                id_col=model.id_col,
                time_col=model.time_col,
                target_col=model.target_col,
            )


In [ ]:
test_pred_insample = model.predict_insample(step_size=horizon)


In [ ]:
test_pred_insample['mean']=test_pred_insample.filter(like='Auto').mean(axis=1)
# test_pred_insample['mean']
# test_pred_insample['mean']
# test_pred_insample.filter(like='Auto').mean(axis=1)
# test_pred_insample.filter(like='Auto').mean(axis=1)

In [ ]:
# test_pred_insample.columns=test_pred_insample.columns.str.replace('Auto','')

In [ ]:
test_data['close']=test_data['y'].copy()


In [ ]:
idx,cut_data=next(iter(test_pred_insample.groupby(['cutoff','symbol'])))
def flatten_preds(idx,cut_data):
    t_off_pred,symb=idx
    pred_cols=cut_data.filter(like='Auto').columns
    cut_data['mean_pred']=cut_data[pred_cols].mean(axis=1)
    cut_data=cut_data.drop(pred_cols,axis=1)
    t_cut=cut_data.T

    t_cut=t_cut.drop('cutoff')
    t_cut=t_cut.drop('ds')
    t_cut=t_cut.drop('y')
    # print(t_cut)
    h_cols=[f'H{i}' for i in range(horizon-1)]
    # print (h_cols)
    t_cut.columns=h_cols


    flat_cols=[f'feature_{model}_{horizon}' for model in t_cut.index for horizon in t_cut.columns]
    pred_values=t_cut.values.flatten()

    flat_df=pd.DataFrame([pred_values],columns=flat_cols)
    id_cols=['ds','symbol']

    flat_df[id_cols]=t_off_pred,symb
    flat_df=  flat_df[id_cols+flat_cols]
    return flat_df

def simulate_forecasts(model,df):
    model.dataset, model.uids, model.last_dates, model.ds = model._prepare_fit(
                df=test_data,
                static_df=None,
                sort_df=model.sort_df,
                predict_only=False,
                id_col=model.id_col,
                time_col=model.time_col,
                target_col=model.target_col,
            )
    
    
    return model.predict_insample(step_size=1)

def prepare_forecast_data(model,test_data):
    
    pred_df= simulate_forecasts(model,test_data)
    flattened_preds=pd.concat([flatten_preds(idx,cut_data) for idx,cut_data in pred_df.groupby(['cutoff','symbol'])])
    return flattened_preds



prepare_forecast_data(model,test_data)

In [ ]:
cut_data
for symb,cut in test_pred_insample.groupby('symbol'):
    cut['mean']=cut.filter(like='Auto').mean(axis=1)

    plt.figure(figsize=(10, 5))
    
    plt.plot(cut['ds'], cut['y'], label='True')
    plt.plot(cut['ds'], cut['mean'] ,label='mean Forecast')
    for model in cut.filter(like='Auto').columns:
        plt.plot(cut['ds'], cut[model], label=f'{model} Forecast')
    # plt.plot(cut['ds'], cut['NBEATS'], label='NBEATS Forecast')
    # plt.plot(cut['ds'], cut['BiTCN'], label='BiTCN Forecast')
    # plt.plot(cut['ds'], cut['TFT'], label='TFT Forecast')
    # plt.axvline(cut['ds'].iloc[-12], color='black', linestyle='--', label='Train-Test Split')
    plt.xlabel('Timestamp [t]')
    plt.ylabel(f'{symb} Price')
    plt.grid()
    plt.legend()

In [ ]:
# StatsForecast.plot(test_pred_insample, engine='matplotlib',time_col='ds',id_col=id_col, max_insample_length=horizon,)
StatsForecast.plot(test_pred_insample, engine='matplotlib',time_col='ds',id_col=id_col,target_col='AutoNBEATS', max_insample_length=horizon,)
